In [1]:
import os
import csv
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from pprint import pprint

data_csv = os.path.join('data','human_trafficking.csv')
data_df=pd.read_csv(data_csv, low_memory = False)
data_df.head()

,yearOfRegistration,Datasource,gender,ageBroad,majorityStatus,majorityStatusAtExploit,majorityEntry,citizenship,meansOfControlDebtBondage,meansOfControlTakesEarnings,...,typeOfSexPrivateSexualServices,typeOfSexConcatenated,isAbduction,RecruiterRelationship,CountryOfExploitation,recruiterRelationIntimatePartner,recruiterRelationFriend,recruiterRelationFamily,recruiterRelationOther,recruiterRelationUnknown
0,2002,Case Management,Female,18--20,Adult,-99,-99,CO,-99,-99,...,-99,-99,-99,-99,-99,0,0,0,0,1
1,2002,Case Management,Female,18--20,Adult,-99,-99,CO,-99,-99,...,-99,-99,-99,-99,-99,0,0,0,0,1
2,2002,Case Management,Female,18--20,Adult,-99,-99,CO,-99,-99,...,-99,-99,-99,-99,-99,0,0,0,0,1
3,2002,Case Management,Female,18--20,Adult,-99,-99,CO,-99,-99,...,-99,-99,-99,-99,-99,0,0,0,0,1
4,2002,Case Management,Female,18--20,Adult,-99,-99,CO,-99,-99,...,-99,-99,-99,-99,-99,0,0,0,0,1


In [2]:
data_df.columns

Index(['yearOfRegistration', 'Datasource', 'gender', 'ageBroad',
       'majorityStatus', 'majorityStatusAtExploit', 'majorityEntry',
       'citizenship', 'meansOfControlDebtBondage',
       'meansOfControlTakesEarnings', 'meansOfControlRestrictsFinancialAccess',
       'meansOfControlThreats', 'meansOfControlPsychologicalAbuse',
       'meansOfControlPhysicalAbuse', 'meansOfControlSexualAbuse',
       'meansOfControlFalsePromises', 'meansOfControlPsychoactiveSubstances',
       'meansOfControlRestrictsMovement', 'meansOfControlRestrictsMedicalCare',
       'meansOfControlExcessiveWorkingHours', 'meansOfControlUsesChildren',
       'meansOfControlThreatOfLawEnforcement',
       'meansOfControlWithholdsNecessities',
       'meansOfControlWithholdsDocuments', 'meansOfControlOther',
       'meansOfControlNotSpecified', 'meansOfControlConcatenated',
       'isForcedLabour', 'isSexualExploit', 'isOtherExploit', 'isSexAndLabour',
       'isForcedMarriage', 'isForcedMilitary', 'isOrganRemova

In [3]:
# year_group = data_df.groupby('yearOfRegistration')
# year_group.count()

In [4]:
#data_df.groupby('CountryOfExploitation').count()

In [5]:
data_df['typeOfLabourConcatenated'].unique()

array(['-99', 'Not specified', 'Other;Not specified', 'Hospitality',
       'Domestic work', 'Domestic work;Not specified', 'Begging',
       'Agriculture', 'Construction', 'Manufacturing',
       'Construction;Not specified', 'Agriculture;Not specified', 'Other',
       'Aquafarming', 'Peddling', 'Domestic work;Other'], dtype=object)

In [6]:
labor_df = data_df[['yearOfRegistration', 'gender', 'ageBroad',
       'majorityStatus', 'majorityStatusAtExploit', 'majorityEntry',
       'citizenship', 'CountryOfExploitation' ,'typeOfLabourConcatenated']]

In [7]:
# removed columns where typeOfLabourConcatendated = -99 (has missing data)
labor_df = labor_df.loc[labor_df['typeOfLabourConcatenated'] != '-99', :]


In [8]:
# replaced Construction;Not specified with Construction
labor_df['typeOfLabourConcatenated'] = labor_df['typeOfLabourConcatenated'].replace('Construction;Not specified', 'Construction')

# replaced Agriculture;Not specified with Agriculture
labor_df['typeOfLabourConcatenated'] = labor_df['typeOfLabourConcatenated'].replace('Agriculture;Not specified', 'Agriculture')

# replace Domestic work;Not specified'& Domestic work;Other'
labor_df['typeOfLabourConcatenated'] = labor_df['typeOfLabourConcatenated'].replace('Domestic work;Not specified', 'Domestic work')
labor_df['typeOfLabourConcatenated'] = labor_df['typeOfLabourConcatenated'].replace('Domestic work;Other', 'Domestic work')

# replace Other and Other;Not specified
labor_df['typeOfLabourConcatenated'] = labor_df['typeOfLabourConcatenated'].replace('Other;Not specified', 'Not specified')
labor_df['typeOfLabourConcatenated'] = labor_df['typeOfLabourConcatenated'].replace('Other', 'Not specified')


In [9]:
labor_df['typeOfLabourConcatenated'].unique()

array(['Not specified', 'Hospitality', 'Domestic work', 'Begging',
       'Agriculture', 'Construction', 'Manufacturing', 'Aquafarming',
       'Peddling'], dtype=object)

In [10]:
labor_df.head()

,yearOfRegistration,gender,ageBroad,majorityStatus,majorityStatusAtExploit,majorityEntry,citizenship,CountryOfExploitation,typeOfLabourConcatenated
1607,2005,Female,0--8,Minor,-99,-99,GH,GH,Not specified
1608,2005,Female,0--8,Minor,-99,-99,GH,GH,Not specified
1609,2005,Female,0--8,Minor,-99,-99,GH,GH,Not specified
1610,2005,Female,0--8,Minor,-99,-99,GH,GH,Not specified
1611,2005,Female,0--8,Minor,-99,-99,GH,GH,Not specified


In [12]:
sex_df = data_df[['yearOfRegistration', 'gender', 'ageBroad',
       'majorityStatus', 'majorityStatusAtExploit', 'majorityEntry',
       'citizenship', 'CountryOfExploitation' , 'typeOfSexConcatenated']]
sex_df = sex_df.loc[sex_df['typeOfSexConcatenated'] != '-99', :]

In [13]:
sex_df['typeOfSexConcatenated'].unique()

array(['Prostitution', 'Pornography', 'Private sexual services'],
      dtype=object)

In [14]:
sex_df.head()

,yearOfRegistration,gender,ageBroad,majorityStatus,majorityStatusAtExploit,majorityEntry,citizenship,CountryOfExploitation,typeOfSexConcatenated
18589,2015,Female,18--20,Adult,Adult,-99,00,US,Prostitution
18590,2015,Female,18--20,Adult,-99,-99,00,US,Prostitution
18591,2015,Female,18--20,Adult,Minor,Adult,00,US,Prostitution
18592,2015,Female,18--20,Adult,-99,-99,00,US,Prostitution
18593,2015,Female,18--20,Adult,Minor,-99,00,US,Prostitution


In [15]:
exploit_df = data_df[['yearOfRegistration', 'gender', 'ageBroad',
       'majorityStatus', 'majorityStatusAtExploit', 'majorityEntry',
       'citizenship', 'CountryOfExploitation' , 'typeOfExploitConcatenated']]
exploit_df = exploit_df.loc[exploit_df['typeOfExploitConcatenated'] != '-99', :]


In [16]:
exploit_df['typeOfExploitConcatenated'].unique()

array(['Sexual exploitation', 'Forced labour', 'Other',
       'Forced labour;Other', 'Slavery and similar practices',
       'Forced marriage',
       'Forced labour;Sexual exploitation;Combined sexual and labour exploitation'],
      dtype=object)

In [17]:
# replace Forced labour;Other with Forced labour
exploit_df['typeOfExploitConcatenated'] = exploit_df['typeOfExploitConcatenated'].replace('Forced labour;Other', 'Forced labour')
exploit_df['typeOfExploitConcatenated'] = exploit_df['typeOfExploitConcatenated'].replace('Forced labour;Sexual exploitation;Combined sexual and labour exploitation', 'Forced labour and Sexual exploitation')


In [18]:
exploit_df.groupby('typeOfExploitConcatenated').count()

,yearOfRegistration,gender,ageBroad,majorityStatus,majorityStatusAtExploit,majorityEntry,citizenship,CountryOfExploitation
typeOfExploitConcatenated,,,,,,,,
Forced labour,8970,8970,8970,8970,8970,8970,8970,8970
Forced labour and Sexual exploitation,78,78,78,78,78,78,78,78
Forced marriage,168,168,168,168,168,168,168,168
Other,7063,7063,7063,7063,7063,7063,7063,7063
Sexual exploitation,15989,15989,15989,15989,15989,15989,15989,15989
Slavery and similar practices,359,359,359,359,359,359,359,359


In [28]:
data_df['meansOfControlConcatenated'] = data_df['meansOfControlConcatenated'].replace('-99', 'unknown')

In [29]:
meansList = ['meansOfControlDebtBondage',
       'meansOfControlTakesEarnings', 'meansOfControlRestrictsFinancialAccess',
       'meansOfControlThreats', 'meansOfControlPsychologicalAbuse',
       'meansOfControlPhysicalAbuse', 'meansOfControlSexualAbuse',
       'meansOfControlFalsePromises', 'meansOfControlPsychoactiveSubstances',
       'meansOfControlRestrictsMovement', 'meansOfControlRestrictsMedicalCare',
       'meansOfControlExcessiveWorkingHours', 'meansOfControlUsesChildren',
       'meansOfControlThreatOfLawEnforcement',
       'meansOfControlWithholdsNecessities',
       'meansOfControlWithholdsDocuments']

for control in meansList:
    data_df[control] = data_df[control].replace(-99, 0)

control_df = data_df[['yearOfRegistration', 'gender', 'ageBroad',
       'majorityStatus', 'majorityStatusAtExploit', 'majorityEntry',
       'citizenship', 'CountryOfExploitation' , 'meansOfControlDebtBondage',
       'meansOfControlTakesEarnings', 'meansOfControlRestrictsFinancialAccess',
       'meansOfControlThreats', 'meansOfControlPsychologicalAbuse',
       'meansOfControlPhysicalAbuse', 'meansOfControlSexualAbuse',
       'meansOfControlFalsePromises', 'meansOfControlPsychoactiveSubstances',
       'meansOfControlRestrictsMovement', 'meansOfControlRestrictsMedicalCare',
       'meansOfControlExcessiveWorkingHours', 'meansOfControlUsesChildren',
       'meansOfControlThreatOfLawEnforcement',
       'meansOfControlWithholdsNecessities',
       'meansOfControlWithholdsDocuments','meansOfControlConcatenated']]

control_df.head()

,yearOfRegistration,gender,ageBroad,majorityStatus,majorityStatusAtExploit,majorityEntry,citizenship,CountryOfExploitation,meansOfControlDebtBondage,meansOfControlTakesEarnings,...,meansOfControlFalsePromises,meansOfControlPsychoactiveSubstances,meansOfControlRestrictsMovement,meansOfControlRestrictsMedicalCare,meansOfControlExcessiveWorkingHours,meansOfControlUsesChildren,meansOfControlThreatOfLawEnforcement,meansOfControlWithholdsNecessities,meansOfControlWithholdsDocuments,meansOfControlConcatenated
0,2002,Female,18--20,Adult,-99,-99,CO,-99,0,0,...,0,0,0,0,0,0,0,0,0,unknown
1,2002,Female,18--20,Adult,-99,-99,CO,-99,0,0,...,0,0,0,0,0,0,0,0,0,unknown
2,2002,Female,18--20,Adult,-99,-99,CO,-99,0,0,...,0,0,0,0,0,0,0,0,0,unknown
3,2002,Female,18--20,Adult,-99,-99,CO,-99,0,0,...,0,0,0,0,0,0,0,0,0,unknown
4,2002,Female,18--20,Adult,-99,-99,CO,-99,0,0,...,0,0,0,0,0,0,0,0,0,unknown


In [32]:
control_df.dtypes

yearOfRegistration                         int64
gender                                    object
ageBroad                                  object
majorityStatus                            object
majorityStatusAtExploit                   object
majorityEntry                             object
citizenship                               object
CountryOfExploitation                     object
meansOfControlDebtBondage                  int64
meansOfControlTakesEarnings                int64
meansOfControlRestrictsFinancialAccess     int64
meansOfControlThreats                      int64
meansOfControlPsychologicalAbuse           int64
meansOfControlPhysicalAbuse                int64
meansOfControlSexualAbuse                  int64
meansOfControlFalsePromises                int64
meansOfControlPsychoactiveSubstances       int64
meansOfControlRestrictsMovement            int64
meansOfControlRestrictsMedicalCare         int64
meansOfControlExcessiveWorkingHours        int64
meansOfControlUsesCh

In [45]:
data_df['RecruiterRelationship'] = data_df['RecruiterRelationship'].replace('-99', 'Not Specified')
recruiterList = ['recruiterRelationIntimatePartner',
       'recruiterRelationFriend', 'recruiterRelationFamily',
       'recruiterRelationOther', 'recruiterRelationUnknown']

for control in recruiterList:
    data_df[control] = data_df[control].replace(-99, 0)

In [52]:
recruiter_df = data_df[['yearOfRegistration', 'gender', 'ageBroad', 'RecruiterRelationship', 'recruiterRelationIntimatePartner',
       'recruiterRelationFriend', 'recruiterRelationFamily',
       'recruiterRelationOther', 'recruiterRelationUnknown']]

In [53]:
recruiter_df.groupby('yearOfRegistration').count()

,gender,ageBroad,RecruiterRelationship,recruiterRelationIntimatePartner,recruiterRelationFriend,recruiterRelationFamily,recruiterRelationOther,recruiterRelationUnknown
yearOfRegistration,,,,,,,,
2002,1026,1026,1026,1026,1026,1026,1026,1026
2003,357,357,357,357,357,357,357,357
2004,223,223,223,223,223,223,223,223
2005,1455,1455,1455,1455,1455,1455,1455,1455
2006,1423,1423,1423,1423,1423,1423,1423,1423
2007,1417,1417,1417,1417,1417,1417,1417,1417
2008,802,802,802,802,802,802,802,802
2009,587,587,587,587,587,587,587,587
2010,943,943,943,943,943,943,943,943
